## 태스크 2: 모델 훈련

이전 태스크에서는 기본 제공 SageMaker XGBoost 알고리즘을 사용했습니다. 이 XGBoost 기본 제공 알고리즘 모드는 사용자의 XGBoost 훈련 스크립트를 통합하지 않으며 입력 데이터세트에서 직접 실행됩니다. 

이 태스크에서는 프레임워크로 XGBoost를 사용합니다. SageMaker 내에서 XGBoost를 프레임워크로 실행하면 자신만의 훈련 스크립트를 사용자 지정할 수 있으므로 유연성이 향상되고 K-폴드 교차 검증과 같은 고급 시나리오에 액세스할 수 있습니다. 

훈련 스크립트를 사용하여 훈련 작업을 실행하는 것 외에도 하이퍼파라미터 튜닝 작업을 실행하여 훈련을 최적화합니다. 하이퍼파라미터는 모델 훈련 중 학습 프로세스에 영향을 주는 상위 파라미터입니다. 모델의 예측 정확도를 최대한 높이려는 경우 하이퍼파라미터 구성을 최적화하거나 하이퍼파라미터 범위를 설정할 수 있습니다. 최적 구성을 찾는 프로세스가 하이퍼파라미터 튜닝입니다. 


### 태스크 2.1: 환경 설정

모델 훈련을 시작하기 전에 필요한 종속 항목을 모두 설치하십시오.


In [ ]:
import pandas as pd
import boto3
import sagemaker
import json
import joblib
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)
from sagemaker.inputs import TrainingInput
from sagemaker.image_uris import retrieve
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

# Setting SageMaker variables
sess = sagemaker.Session()
region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sagemaker_role = sagemaker.get_execution_role()
sagemaker_client = boto3.client("sagemaker")

 다음으로는 이전 태스크에서와 마찬가지로 훈련 작업에서 입력으로 사용하는 훈련 및 검증 경로를 구성합니다.

In [ ]:
# define the bucket and prefix
#read_bucket = "<LAB_BUCKET>"
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        read_bucket = buckets.name
#read_bucket = "labdatabucket-us-west-2-110972467"
read_prefix = "scripts/data" 

# Setting S3 location for read and write operations
train_data_key = f"{read_prefix}/train/adult_data_processed_train_wheaders.csv"
test_data_key = f"{read_prefix}/test/adult_data_processed_test_wheaders.csv"
validation_data_key = f"{read_prefix}/validation/adult_data_processed_validation_wheaders.csv"

write_bucket = read_bucket
write_prefix = "script-mode/data"

model_key = f"{write_prefix}/model"
output_key = f"{write_prefix}/output"

train_data_uri = f"s3://{read_bucket}/{train_data_key}"
validation_data_uri = f"s3://{read_bucket}/{validation_data_key}"
test_data_uri = f"s3://{read_bucket}/{test_data_key}"
model_uri = f"s3://{write_bucket}/{model_key}"
output_uri = f"s3://{write_bucket}/{output_key}"
estimator_output_uri = f"s3://{write_bucket}/{write_prefix}/training_jobs"
bias_report_output_uri = f"s3://{write_bucket}/{write_prefix}/clarify-output/bias"
explainability_report_output_uri = f"s3://{write_bucket}/{write_prefix}/clarify-output/explainability"

### 태스크 2.2: 추정기 객체 구성

튜닝 작업 구성 

In [ ]:
training_job_name_prefix = "xgbtrain"
tuning_job_name_prefix = "xgbtune" 
xgb_model_name = "xgb-script-mode-model"
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

정적 하이퍼파라미터, 하이퍼파라미터 범위 및 추정기 객체를 구성합니다. 

태스크 1에서와 달리 이 스크립트 모드 태스크에서는 훈련 작업용 사용자 지정 스크립트를 정의합니다. 아래에서 SageMaker 추정기를 정의할 때는 사용자 지정된 Python 스크립트를 진입점으로 사용합니다. 사용자 지정 xgboost_train.py 파일을 열어 보십시오.

사용자 지정 xgboost_train.py 파일에는 교차 검증 기법 수행 섹션이 있습니다. 그런데 기본 제공 SageMaker XGBoost 훈련 알고리즘으로는 이 작업을 수행할 수 없습니다.

그리고 다음 셀에서 정의한 하이퍼파라미터 범위는 첫 번째 태스크의 XGBoost 기본 제공 훈련에 사용했던 것과 같은 정적 하이퍼파라미터입니다. 단, 이번에는 튜너가 최적의 최종 목표 지표가 적용된 조합을 찾기 위해 이러한 하이퍼파라미터의 여러 값을 사용해 보도록 범위를 정의합니다. 


In [ ]:
# Set static hyperparameters that will not be tuned
static_hyperparams = {  
                        "eval_metric" : "auc",
                        "objective": "binary:logistic",
                        "num_round": "5"
                      }

# hyperparameter ranges that will be tuned 
hyperparameter_ranges = {
    "max_depth": IntegerParameter(6, 9),
    "eta": ContinuousParameter(0.01, 0.03),
    "gamma": ContinuousParameter(0.5, 0.9),
    "min_child_weight": ContinuousParameter(0.5, 0.9),
    "subsample": ContinuousParameter(0.2, 0.5)
}

# XGBoost Estimator 
xgb_estimator = XGBoost(
                        entry_point="xgboost_train.py",
                        output_path=estimator_output_uri,
                        code_location=estimator_output_uri,
                        hyperparameters=static_hyperparams,
                        role=sagemaker_role,
                        instance_count=train_instance_count,
                        instance_type=train_instance_type,
                        framework_version="1.7-1",
                        base_job_name=training_job_name_prefix
                    )

이제 앞서 정의한 하이퍼파라미터 범위와 XGBoost 추정기를 사용할 튜너 객체를 생성합니다. 

In [ ]:
objective_metric_name = "validation:auc"

# Setting up tuner object
tuner_config_dict = {
                     "estimator" : xgb_estimator,
                     "max_jobs" : 6,
                     "max_parallel_jobs" : 3,    
                     "objective_metric_name" : objective_metric_name,
                     "hyperparameter_ranges" : hyperparameter_ranges,
                     "base_tuning_job_name" : tuning_job_name_prefix,
                     "strategy" : "Random"
                    }
tuner = HyperparameterTuner(**tuner_config_dict)

튜닝 작업용 입력 채널 설정 및 튜너 작업 실행 

In [ ]:
s3_input_train = TrainingInput(s3_data="s3://{}/{}".format(read_bucket, train_data_key), content_type="csv", s3_data_type="S3Prefix")
s3_input_validation = (TrainingInput(s3_data="s3://{}/{}".format(read_bucket, validation_data_key), content_type="csv", s3_data_type="S3Prefix"))

tuner.fit(inputs={"train": s3_input_train, "validation": s3_input_validation}, include_cls_metadata=False)
tuner.wait()

<i aria-hidden="true" class="fas fa-clipboard-check" style="color:#18ab4b"></i> **Expected output:** 추정기 및 하이퍼파라미터 구성이 올바르고 하이퍼파라미터 튜닝 작업이 올바르게 완료된 경우 다음 출력이 표시됩니다.

```plain
************************
**** 출력 예시 ****
************************

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config(이 추정기와 관련하여 완료된 훈련 작업이 없습니다. 이 추정기가 워크플로 구성 빌드에만 사용되는지 확인하십시오.)
...................................!
!
```

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i> **Note:** 훈련을 실행하려면 약 3~4분이 소요됩니다.


성능의 내림차순으로 튜닝 결과 요약을 실행합니다. 결과에서는 가장 높은 목표 값이 맨 위에 표시됩니다.  

In [ ]:
df_tuner = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.job_name).dataframe()
df_tuner = df_tuner[df_tuner["FinalObjectiveValue"]>-float('inf')].sort_values("FinalObjectiveValue", ascending=False)
df_tuner

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 실습을 마무리합니다.